In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

file_path = "../mortalidade_2024.csv"
df = pd.read_csv(file_path, sep=";", encoding="latin1")

df.head()


,DATA_ÓBITO,NOME_REGIAO,NOME_REGIAO_SAUDE,MUNICIPIO,FAIXA_ETÁRIA,ESCOLARIDADE,SEXO,RACA,CID,ACIDENTES,QUANTIDADE
0,2023-12-31,CARIRI,18ª REGIÃO IGUATU,ACOPIARA,80 ou mais,Ignorado,FEMININO,Branca,Causas externas de morbidade e mortalidade,Outras causas,1
1,2023-12-31,CARIRI,18ª REGIÃO IGUATU,MOMBACA,20 a 29 anos,Fundamental I,MASCULINO,Não Informado,Causas externas de morbidade e mortalidade,Outras causas,1
2,2023-12-31,CARIRI,19ª REGIÃO BREJO SANTO,AURORA,80 ou mais,Fundamental I,MASCULINO,Parda,Sint sinais e achad anorm ex clín e laborat,Outras causas,1
3,2023-12-31,CARIRI,19ª REGIÃO BREJO SANTO,BREJO SANTO,80 ou mais,Sem escolaridade,MASCULINO,Parda,Causas externas de morbidade e mortalidade,Outras causas,1
4,2023-12-31,CARIRI,19ª REGIÃO BREJO SANTO,JATI,80 ou mais,Sem escolaridade,FEMININO,Parda,Doenças do aparelho digestivo,Outras causas,1


In [12]:
# print a table grouping the number of deaths by CID, ordered by the number of deaths
print(df.groupby("CID").size().sort_values(ascending=False))

CID
Doenças do aparelho circulatório                 14960
Neoplasias Benigna Malignas(tumores)             10074
Doenças do aparelho respiratório                  8219
Causas externas de morbidade e mortalidade        7925
Doenças do aparelho digestivo                     3228
Algumas doenças infecciosas e parasitárias        3012
Doenças endócrinas nutricionais e metabólicas     2706
Doenças do sistema nervoso                        2443
Algumas afec originadas no período perinatal      1716
Doenças do aparelho geniturinário                 1708
Sint sinais e achad anorm ex clín e laborat       1691
Transtornos mentais e comportamentais              951
Malf cong deformid e anomalias cromossômicas       511
Doenças da pele e do tecido subcutâneo             490
Doenças sist osteomuscular e tec conjuntivo        319
Doenças sangue órgãos hemat e transt imunitár      284
Gravidez parto e puerpério                          48
Códigos para propósitos especiais                   24
Doença

In [46]:
# create a new boolean column if a death was caused by a tumor
new_column_name = "MortePorTumor"
new_column_value = "(tumores)"

df[new_column_name] = df["CID"].str.contains(new_column_value).astype(int)

df.head()

/var/folders/hn/w49nyqc159bcvzxhg2vmzfv40000gn/T/ipykernel_21340/3025663663.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[new_column_name] = df["CID"].str.contains(new_column_value).astype(int)


,DATA_ÓBITO,NOME_REGIAO,NOME_REGIAO_SAUDE,MUNICIPIO,FAIXA_ETÁRIA,ESCOLARIDADE,SEXO,RACA,CID,ACIDENTES,QUANTIDADE,MortePorTumor
0,364,0,8,3,10,2,0,1,Causas externas de morbidade e mortalidade,2,0,0
1,364,0,8,114,3,0,1,3,Causas externas de morbidade e mortalidade,2,0,0
2,364,0,9,19,10,0,1,4,Sint sinais e achad anorm ex clín e laborat,2,0,0
3,364,0,9,30,10,4,1,4,Causas externas de morbidade e mortalidade,2,0,0
4,364,0,9,96,10,4,0,4,Doenças do aparelho digestivo,2,0,0


In [47]:
print(df.groupby(new_column_name).size().sort_values(ascending=False))

MortePorTumor
0    50258
1    10074
dtype: int64


In [48]:
feature_names = df.columns.tolist()
feature_names.remove("CID")
feature_names.remove(new_column_name)

target_names = (1, 0)

print(feature_names)
print(target_names)

['DATA_ÓBITO', 'NOME_REGIAO', 'NOME_REGIAO_SAUDE', 'MUNICIPIO', 'FAIXA_ETÁRIA', 'ESCOLARIDADE', 'SEXO', 'RACA', 'ACIDENTES', 'QUANTIDADE']
(1, 0)


In [49]:
# transform all entries in the feature columns to numbers
for feature_name in feature_names:
    df[feature_name] = pd.Categorical(df[feature_name]).codes

df.head()

,DATA_ÓBITO,NOME_REGIAO,NOME_REGIAO_SAUDE,MUNICIPIO,FAIXA_ETÁRIA,ESCOLARIDADE,SEXO,RACA,CID,ACIDENTES,QUANTIDADE,MortePorTumor
0,364,0,8,3,10,2,0,1,Causas externas de morbidade e mortalidade,2,0,0
1,364,0,8,114,3,0,1,3,Causas externas de morbidade e mortalidade,2,0,0
2,364,0,9,19,10,0,1,4,Sint sinais e achad anorm ex clín e laborat,2,0,0
3,364,0,9,30,10,4,1,4,Causas externas de morbidade e mortalidade,2,0,0
4,364,0,9,96,10,4,0,4,Doenças do aparelho digestivo,2,0,0


In [50]:
from sklearn.model_selection import train_test_split

X = df[feature_names].to_numpy()
y = df[new_column_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(X_train, X_test)

[[226   1  10 ...   4   2   0]
 [273   3  20 ...   1   0   0]
 [279   4   5 ...   1   2   0]
 ...
 [358   0   8 ...   4   0   0]
 [266   1  14 ...   4   2   0]
 [ 24   1  10 ...   4   2   0]] [[227   1  10 ...   4   2   0]
 [193   4   3 ...   4   2   0]
 [316   1  10 ...   4   2   0]
 ...
 [332   4   3 ...   4   2   0]
 [142   1  13 ...   1   2   0]
 [232   1  15 ...   1   2   0]]


In [51]:
from sklearn.linear_model import LogisticRegression

print(X_train)
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

[[226   1  10 ...   4   2   0]
 [273   3  20 ...   1   0   0]
 [279   4   5 ...   1   2   0]
 ...
 [358   0   8 ...   4   0   0]
 [266   1  14 ...   4   2   0]
 [ 24   1  10 ...   4   2   0]]


/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [52]:
y_pred = log_reg.predict(X_test)

In [53]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8300072929788503
